In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model
from keras.losses import mean_absolute_error
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.utils import plot_model

from LRF import LRFinder
from SGDR import SGDRScheduler
from xception import Xception

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'
npy_img_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data\\training_images_batches'
architecture_plot_save_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'

img_width, img_height = 299., 299.
rx = 2.1404
ry = 1.61

In [3]:
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
epochs = 1000
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/xception_lrf_submission.h5"

In [4]:
base_model = Xception(include_top=False, weights=None, input_shape = (299, 299, 3), classes = 4)

In [5]:
# for layer in base_model.layers[:-3]:
#     layer.trainable= False

In [6]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(4, activation="relu")(x)

In [7]:
model = Model(input = base_model.input, output = predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [8]:
# model.load_weights("X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_lrf_submission.h5")

In [9]:
def mIoU(labels,predictions):
    pred_x1=predictions[:,0]
    pred_x2=predictions[:,1]
    pred_y1=predictions[:,2]
    pred_y2=predictions[:,3]
    label_x1=labels[:,0]
    label_x2=labels[:,1]
    label_y1=labels[:,2]
    label_y2=labels[:,3]
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [10]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean:0", shape=(), dtype=float64)
0.5714285714285714


#### Load best model weights from here, if required.

In [11]:
augment_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
augment_val_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv'))

In [13]:
augment_df.head()

image_name     x1     x2     y1     y2
0  [v]JPEG_20160621_121359_100048151077.png  256.0  461.0   12.0  406.0
1              [v]1474638122306DSC06451.png   19.0  590.0  168.0  422.0
2       [v]147772181322620161028_165831.png    4.0  536.0  128.0  319.0
3       [v]147771810022720161028_154644.png   53.0  556.0    0.0  449.0
4    JPEG_20161205_113835_1000778288816.png   57.0  536.0   82.0  293.0

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_dataframe(dataframe=augment_df,
                                            directory=os.path.join(base_dir, 'augument_data', 'resized_training_images'),
                                            x_col='image_name',
                                            y_col=['x1', 'x2', 'y1', 'y2'],
                                            class_mode="other",
                                                  target_size = (299, 299),
                                                  batch_size=8,
                                                  shuffle = True)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=augment_val_df,
                                            directory=os.path.join(base_dir, 'augument_data', 'training_images'),
                                            x_col='image_name',
                                            y_col=['x1', 'x2', 'y1', 'y2'],
                                            class_mode="other",
                                            target_size = (299, 299),
                                            batch_size=8,
                                            shuffle = True)

Found 68000 images.
Found 2000 images.


In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [14]:
model.compile(loss = mean_absolute_error, optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=[mIoU])

In [15]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [16]:
def show_img_with_rect(img, lab):
    x1 = int(lab[0]/2.1404)
    x2 = int(lab[1]/2.1404)
    y1 = int(lab[2]/1.61)
    y2 = int(lab[3]/1.61) #
    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [17]:
# '''
#     lrf = LRFinder(minimum=3e-6, maximum=3e-4, step_size=(train_generator.n//train_generator.batch_size), n_epochs=4)
#     model.fit(x_train, y_train, callbacks=[lrf, ])
#     lrf.lr_graph()
#     lrf.loss_graph()
# '''
# lrf = LRFinder(minimum=3e-6, maximum=3e-4, step_size=(train_generator.n//train_generator.batch_size), n_epochs=2)

In [18]:
# model.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=valid_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=2,
#                     verbose=1,
#                     callbacks=[checkpoint, lrf],
#                     shuffle=True)

In [19]:
# lrf.lr_graph()

In [20]:
# lrf.loss_graph()

In [21]:
schedule = SGDRScheduler(min_lr=3e-5,
                         max_lr=1e-3,
                         steps_per_epoch=STEP_SIZE_TRAIN,
                         lr_decay=0.9,
                         cycle_length=2,
                         mult_factor=1.5)

In [22]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1000,
                    verbose=1,
                    callbacks=[checkpoint, schedule],
                    shuffle=True)

Epoch 1/1000
8500/8500 [==============================] - 2586s 304ms/step - loss: 19.9729 - mIoU: 0.7977 - val_loss: 11.4349 - val_mIoU: 0.8789

Epoch 00001: val_loss improved from inf to 11.43487, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/xception_lrf_submission.h5
Epoch 2/1000
8500/8500 [==============================] - 2566s 302ms/step - loss: 9.5314 - mIoU: 0.8966 - val_loss: 7.7935 - val_mIoU: 0.9153

Epoch 00002: val_loss improved from 11.43487 to 7.79353, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/xception_lrf_submission.h5
Epoch 3/1000
8500/8500 [==============================] - 2567s 302ms/step - loss: 11.1443 - mIoU: 0.8790 - val_loss: 11.3839 - val_mIoU: 0.8781

Epoch 00003: val_loss did not improve from 7.79353
Epoch 4/1000
8500/8500 [==============================] - 2566s 302ms/step - loss: 8.4734 - mIoU: 0.9070 - val_loss: 7.9643 - val_mIoU: 0.9123

Epoch 00004: val_loss did not

KeyboardInterrupt: 

In [23]:
model.save('xception_lrf_trained_val95.h5')